In [1]:
# Installs the necessary Python and system libraries
try:
    from easypip import easyimport, easyinstall, is_notebook
except ModuleNotFoundError as e:
    get_ipython().run_line_magic("pip", "install easypip")
    from easypip import easyimport, easyinstall, is_notebook

easyinstall("bbrl>=0.2.2")
easyinstall("swig")
easyinstall("bbrl_gymnasium>=0.2.0")
easyinstall("bbrl_gymnasium[box2d]")
easyinstall("bbrl_gymnasium[classic_control]")
easyinstall("tensorboard")
easyinstall("moviepy")
easyinstall("box2d-kengz")


[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import sys
from pathlib import Path
import math

from moviepy.editor import ipython_display as video_display
import time
from tqdm.auto import tqdm
from typing import Tuple, Optional
from functools import partial

from omegaconf import OmegaConf
import torch
import bbrl_gymnasium

import copy
from abc import abstractmethod, ABC
import torch.nn as nn
import torch.nn.functional as F
from time import strftime
OmegaConf.register_new_resolver(
    "current_time", lambda: strftime("%Y%m%d-%H%M%S"), replace=True
)
from bbrl.agents.gymnasium import GymAgent, ParallelGymAgent, make_env, record_video

In [3]:
from datetime import datetime
from datetime import date

import base64
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

import gymnasium
from gymnasium import logger as gymlogger
# from gym.wrappers import Monitor # deprecated 2023 - https://stackoverflow.com/questions/71520568/importerror-cannot-import-name-monitor-from-gym-wrappers
from gymnasium.wrappers.record_video import RecordVideo
gymlogger.set_level(40) #error only
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

print("\n",date.today(), datetime.now().strftime("%H:%M:%S"),"GMT") # timestamp is greenwich time
print("OK.")

def show_video(loop=True, num=0):
    mp4list = glob.glob(f'videoTest/rl-video-episode-{num}.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        if loop == True:
            ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:videoTest/mp4;base64,{0}" type="video/mp4" />
                 </video>'''.format(encoded.decode('ascii'))))
        else:
            ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                    controls style="height: 400px;">
                    <source src="data:videoTest/mp4;base64,{0}" type="video/mp4" />
                 </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")
    
def wrap_env(env):
    env = RecordVideo(env, './videoTest',  episode_trigger = lambda episode_number: True) # !!! 2023
    env.reset() # !!! 2023
    #env = Monitor(env, './video', force=True)
    return env


 2024-01-10 16:05:55 GMT
OK.


In [33]:
"""
Continuous action version of the classic cart-pole system implemented by Rich
Sutton et al.
"""
import math

import numpy as np

from gymnasium.envs.classic_control.cartpole import CartPoleEnv
from gymnasium import logger, spaces
from gymnasium.wrappers import TimeLimit

import sys
sys.modules[__name__]

class ContinuousEnvCACLA(CartPoleEnv):
    """Continuous version  of the CartPole-v1 environment"""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.min_action = -1.0
        self.max_action = 1.0
        self.action_space = spaces.Box(
            self.min_action, self.max_action, shape=(1,), dtype=np.float64
        )

    def step(self, action):
        if action > self.max_action:
            action = np.array(self.max_action)
        elif action < self.min_action:
            action = np.array(self.min_action)
        assert self.state is not None, "Call reset before using step method."

        x, x_dot, theta, theta_dot = self.state
        force = self.force_mag * float(action)

        costheta = math.cos(theta)
        sintheta = math.sin(theta)

        # For the interested reader:
        # https://coneural.org/florian/papers/05_cart_pole.pdf
        temp = (
            force + self.polemass_length * theta_dot**2 * sintheta
        ) / self.total_mass
        thetaacc = (self.gravity * sintheta - costheta * temp) / (
            self.length * (4.0 / 3.0 - self.masspole * costheta**2 / self.total_mass)
        )
        xacc = temp - self.polemass_length * thetaacc * costheta / self.total_mass

        if self.kinematics_integrator == "euler":
            x = x + self.tau * x_dot
            x_dot = x_dot + self.tau * xacc
            theta = theta + self.tau * theta_dot
            theta_dot = theta_dot + self.tau * thetaacc
        else:  # semi-implicit euler
            x_dot = x_dot + self.tau * xacc
            x = x + self.tau * x_dot
            theta_dot = theta_dot + self.tau * thetaacc
            theta = theta + self.tau * theta_dot

        self.state = (x, x_dot, theta, theta_dot)

        terminated = bool(
            x < -self.x_threshold
            or x > self.x_threshold
            or theta < -self.theta_threshold_radians
            or theta > self.theta_threshold_radians
        )

        if not terminated:
            reward = 1.0
        elif self.steps_beyond_terminated is None:
            # Pole just fell!
            self.steps_beyond_terminated = 0
            reward = -1.0
        else:
            if self.steps_beyond_terminated == 0:
                logger.warn(
                    "You are calling 'step()' even though this "
                    "environment has already returned terminated = True. You "
                    "should always call 'reset()' once you receive 'terminated = "
                    "True' -- any further steps are undefined behavior."
                )
            self.steps_beyond_terminated += 1
            reward = 0

        if self.render_mode == "human":
            self.render()
        return np.array(self.state, dtype=np.float32), reward, terminated, False, {}

try:
  gymnasium.envs.register(
      id='CartpoleEnvCacla',
      entry_point='__main__:ContinuousEnvCACLA',
      max_episode_steps=500
  )
except:
    print("Except")
    pass



In [34]:
import torch.nn
import numpy as np
from torch.optim import SGD, Adam
class Gav(nn.Module):
    def __init__(self, in_dim, h_dim, activation=nn.Tanh, discount_factor=0.95, gaussian_noise=0.01, lrate = 0.01, decay=0.99, exploration="gaussian"):
        super(Gav,self).__init__()
        self.critic = nn.Sequential(
            nn.Linear(in_dim+1, h_dim),
            activation(),
            nn.Linear(h_dim, 1)
        )
        self.critic_optim = SGD(self.critic.parameters(), lr=lrate)
        
        self.actor = nn.Sequential(
            nn.Linear(in_dim, h_dim),
            activation(),
            nn.Linear(h_dim, 1),
            activation()
        )
        self.actor_optim = SGD(self.actor.parameters(), lr=lrate)
        
        self.discount_factor = discount_factor
        self.gaussian_noise = gaussian_noise
        self.e = 1
        self.prob_update = 1
        self.decay_rate = decay
        self.exploration = exploration
    
    def forward(self, x):
        x = torch.from_numpy(x).unsqueeze(0)
        # le critic prend aussi l'action
        act = self.actor(x)
        # gaussian exploration
        if self.exploration=="gaussian": act = self.sample(act)
        # egreedy exploration
        elif self.exploration=="egreedy" and torch.rand(1) < self.e: act = self.greed()
        x = torch.hstack((x,act))
        return self.critic(x), act
    
    def sample(self, x):
        return (x + np.random.normal(0,self.gaussian_noise))
    
    # nombre entre -1 et 1
    def greed(self):
        return (2 * torch.rand(1) - 1).unsqueeze(0)
    
    def decaye(self):
        self.e*=0.999
        self.e = max(0.01,self.e)
    
    def decay(self):
        self.prob_update*= self.decay_rate
        self.prob_update = max(0.01, self.prob_update)

def test(eval_env, model, n_test = 10):
    cum_reward = 0
    with torch.no_grad():
        for _ in range(n_test):
            done, truncated = False, False
            obs0,_ = eval_env.reset()
            while not done and not truncated:
                _, a0 = model(obs0)  
                obs0, reward, done,truncated,_ = eval_env.step(a0) 
                cum_reward += reward
    return cum_reward/n_test
    
def train(eval_env, train_env, model, eval_interval=5000, step_max=100000, exploration="gaussian"):
    it = 0
    obs,_ = train_env.reset()
    p = 1
    scores = np.array([0])
    while it < step_max:
        it += 1
        model.critic_optim.zero_grad()
            
        q0, action = model(obs)
        
        obs, reward, done, truncated, reste = train_env.step(action+np.random.normal(0,0.1)) 
        reward = model.sample(reward+np.random.normal(0,0.1))
        q1, _ = model(obs)
        delta = reward + model.discount_factor*q1.detach() - q0
        (delta**2).backward()
        model.critic_optim.step()
        if torch.rand(1) > model.prob_update: 
            model.actor_optim.zero_grad()
            (-model(obs)[0]).backward()
            model.actor_optim.step()
        model.decay()
        # pour diminuer exponentiellement
        model.decaye()
        
        if done or truncated: obs,_ = train_env.reset()
        if it % eval_interval == 0:
            perf = test(eval_env, model)
            print(f'{it = } | reward {perf}')
            scores = np.hstack((scores,perf))
    return scores

In [ ]:

# Initialisation
n_runs = 20
final = []
explo = "gaussian"
for run in range(n_runs):
    train_env = gymnasium.make('CartpoleEnvCacla')
    train_env = TimeLimit(train_env, max_episode_steps=500)
    eval_env = gymnasium.make('CartpoleEnvCacla')
    eval_env = TimeLimit(eval_env, max_episode_steps=500)
    print(f'======= {run = } / {n_runs-1} =========')
    gav = Gav(in_dim=4, h_dim=12, 
              discount_factor=0.95, 
              gaussian_noise=0.1,exploration =explo)
    scores = train(eval_env, train_env, gav, eval_interval=1024, step_max=102400)
    print(scores.shape)
    final.append(scores)

final = np.array(final)
np.savetxt("gav"+explo+".txt",final)
    

======= run = 0 / 19 =========
it = 1024 | reward 15.2
it = 2048 | reward 56.7
it = 3072 | reward 66.1
it = 4096 | reward 70.7
it = 5120 | reward 67.9
it = 6144 | reward 82.1
it = 7168 | reward 74.0
it = 8192 | reward 78.2
it = 9216 | reward 487.4
it = 10240 | reward 293.3
it = 11264 | reward 282.1
it = 12288 | reward 308.7
it = 13312 | reward 492.0
it = 14336 | reward 253.0
it = 15360 | reward 363.5
it = 16384 | reward 439.3
it = 17408 | reward 386.4
it = 18432 | reward 452.4
it = 19456 | reward 500.0
it = 20480 | reward 140.0
it = 21504 | reward 348.0
it = 22528 | reward 273.1
it = 23552 | reward 185.0
it = 24576 | reward 224.2
it = 25600 | reward 424.3
it = 26624 | reward 328.1
it = 27648 | reward 425.0
it = 28672 | reward 482.9
it = 29696 | reward 355.4
it = 30720 | reward 417.3
it = 31744 | reward 425.4
it = 32768 | reward 422.9
it = 33792 | reward 460.8
it = 34816 | reward 486.8
it = 35840 | reward 488.6
it = 36864 | reward 500.0
it = 37888 | reward 500.0
it = 38912 | reward 500.

it = 16384 | reward 484.4
it = 17408 | reward 203.4
it = 18432 | reward 229.8
it = 19456 | reward 95.5
it = 20480 | reward 451.0
it = 21504 | reward 282.3
it = 22528 | reward 193.8
it = 23552 | reward 425.8
it = 24576 | reward 402.3
it = 25600 | reward 319.6
it = 26624 | reward 230.1
it = 27648 | reward 218.0
it = 28672 | reward 385.0
it = 29696 | reward 152.2
it = 30720 | reward 131.2
it = 31744 | reward 144.2
it = 32768 | reward 147.5
it = 33792 | reward 261.4
it = 34816 | reward 131.9
it = 35840 | reward 333.4
it = 36864 | reward 244.0
it = 37888 | reward 284.1
it = 38912 | reward 143.2
it = 39936 | reward 158.0
it = 40960 | reward 29.4
it = 41984 | reward 479.6
it = 43008 | reward 173.8
it = 44032 | reward 125.1
it = 45056 | reward 80.8
it = 46080 | reward 131.0
it = 47104 | reward 102.3
it = 48128 | reward 145.6
it = 49152 | reward 129.8
it = 50176 | reward 220.9
it = 51200 | reward 195.2
it = 52224 | reward 252.9
it = 53248 | reward 220.6
it = 54272 | reward 244.9
it = 55296 | re

it = 30720 | reward 489.2
it = 31744 | reward 500.0
it = 32768 | reward 490.7
it = 33792 | reward 495.4
it = 34816 | reward 500.0
it = 35840 | reward 441.4
it = 36864 | reward 500.0
it = 37888 | reward 500.0
it = 38912 | reward 500.0
it = 39936 | reward 421.3
it = 40960 | reward 480.6
it = 41984 | reward 500.0
it = 43008 | reward 500.0
it = 44032 | reward 500.0
it = 45056 | reward 365.6
it = 46080 | reward 450.3
it = 47104 | reward 500.0
it = 48128 | reward 500.0
it = 49152 | reward 500.0
it = 50176 | reward 500.0
it = 51200 | reward 500.0
it = 52224 | reward 500.0
it = 53248 | reward 500.0
it = 54272 | reward 500.0
it = 55296 | reward 500.0
it = 56320 | reward 500.0
it = 57344 | reward 500.0
it = 58368 | reward 500.0
it = 59392 | reward 500.0
it = 60416 | reward 500.0
it = 61440 | reward 500.0
it = 62464 | reward 500.0
it = 63488 | reward 500.0
it = 64512 | reward 500.0
it = 65536 | reward 500.0
it = 66560 | reward 500.0
it = 67584 | reward 500.0
it = 68608 | reward 500.0
it = 69632 |

it = 46080 | reward 500.0
it = 47104 | reward 500.0
it = 48128 | reward 500.0
it = 49152 | reward 500.0
it = 50176 | reward 500.0
it = 51200 | reward 500.0
it = 52224 | reward 499.8
it = 53248 | reward 484.4
it = 54272 | reward 500.0
it = 55296 | reward 490.3
it = 56320 | reward 453.2
it = 57344 | reward 500.0
it = 58368 | reward 500.0
it = 59392 | reward 487.7
it = 60416 | reward 499.3
it = 61440 | reward 404.4
it = 62464 | reward 462.1
it = 63488 | reward 499.1
it = 64512 | reward 493.3
it = 65536 | reward 458.7
it = 66560 | reward 479.6
it = 67584 | reward 474.0
it = 68608 | reward 448.7
it = 69632 | reward 465.6
it = 70656 | reward 485.5
it = 71680 | reward 358.4
it = 72704 | reward 451.4
it = 73728 | reward 500.0
it = 74752 | reward 356.3
it = 75776 | reward 430.0
it = 76800 | reward 463.5
it = 77824 | reward 350.3
it = 78848 | reward 350.1
it = 79872 | reward 441.8
it = 80896 | reward 329.8
it = 81920 | reward 412.1
it = 82944 | reward 425.9
it = 83968 | reward 457.4
it = 84992 |

it = 60416 | reward 204.9
it = 61440 | reward 140.5
it = 62464 | reward 220.7
it = 63488 | reward 244.4
it = 64512 | reward 150.8
it = 65536 | reward 118.3
it = 66560 | reward 120.8
it = 67584 | reward 111.8
it = 68608 | reward 132.9
it = 69632 | reward 104.1
it = 70656 | reward 118.9
it = 71680 | reward 99.5
it = 72704 | reward 111.0
it = 73728 | reward 41.5
it = 74752 | reward 24.0
it = 75776 | reward 256.3
it = 76800 | reward 162.7
it = 77824 | reward 227.0
it = 78848 | reward 292.4
it = 79872 | reward 233.1
it = 80896 | reward 171.3
it = 81920 | reward 295.2
it = 82944 | reward 267.0
it = 83968 | reward 402.0
it = 84992 | reward 138.5
it = 86016 | reward 299.5
it = 87040 | reward 275.8
it = 88064 | reward 371.9
it = 89088 | reward 188.9
it = 90112 | reward 303.5
it = 91136 | reward 323.9
it = 92160 | reward 274.1
it = 93184 | reward 232.8
it = 94208 | reward 200.3
it = 95232 | reward 251.0
it = 96256 | reward 158.5
it = 97280 | reward 173.4
it = 98304 | reward 180.1
it = 99328 | re

In [25]:
rec_env = wrap_env(TimeLimit(ContinuousEnvCACLA(render_mode='rgb_array'),max_episode_steps=500))
observation, _ = rec_env.reset()
maxEvaluations = 2
evaluation = 0
score = []
total = 0
while evaluation < maxEvaluations:
    rec_env.render()
    _, a0 = gav(observation)  
    action = a0
    observation, reward, done, truncated, _ = rec_env.step(action) 
    total += reward
    if done or truncated:
      score.append(total)
      total = 0
      evaluation = evaluation + 1
      observation, _ = rec_env.reset()
rec_env.close()
score

Moviepy - Building video /users/nfs/Etu3/28604113/IAR/PROJET_IAR/ddpg_gav/videoTest/rl-video-episode-0.mp4.
Moviepy - Writing video /users/nfs/Etu3/28604113/IAR/PROJET_IAR/ddpg_gav/videoTest/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /users/nfs/Etu3/28604113/IAR/PROJET_IAR/ddpg_gav/videoTest/rl-video-episode-0.mp4
Moviepy - Building video /users/nfs/Etu3/28604113/IAR/PROJET_IAR/ddpg_gav/videoTest/rl-video-episode-1.mp4.
Moviepy - Writing video /users/nfs/Etu3/28604113/IAR/PROJET_IAR/ddpg_gav/videoTest/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /users/nfs/Etu3/28604113/IAR/PROJET_IAR/ddpg_gav/videoTest/rl-video-episode-1.mp4
Moviepy - Building video /users/nfs/Etu3/28604113/IAR/PROJET_IAR/ddpg_gav/videoTest/rl-video-episode-2.mp4.
Moviepy - Writing video /users/nfs/Etu3/28604113/IAR/PROJET_IAR/ddpg_gav/videoTest/rl-video-episode-2.mp4



Moviepy - Done !
Moviepy - video ready /users/nfs/Etu3/28604113/IAR/PROJET_IAR/ddpg_gav/videoTest/rl-video-episode-2.mp4


[500.0, 500.0]

In [26]:
show_video(num=0)